This code is to test the multi-view graph neural network-transformer 

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import scipy.sparse as sp



def generate_fnirs_adj():
    matrix = sp.csr_matrix((52, 52), dtype=int)
    for i in range(10):
        if i > 0 and i < 9:
            matrix[i, i-1] = 1
            matrix[i, i+1] = 1
        elif i == 0:
            matrix[i, i+1] = 1
        else:
            matrix[i, i-1] = 1
        matrix[i, i+10] = 1
        matrix[i, i+11] = 1

    for i in range(10, 21):
        if i > 10 and i < 20:
            matrix[i, i-11] = 1
            matrix[i, i-10] = 1
            matrix[i, i-1] = 1
            matrix[i, i+1] = 1
            matrix[i, i+10] = 1
            matrix[i, i+11] = 1
        if i == 10:
            matrix[i, i-10] = 1
            matrix[i, i+1] = 1
            matrix[i, i+11] = 1
        if i == 20:
            matrix[i, i-11] = 1
            matrix[i, i-1] = 1
            matrix[i, i+10] = 1

    for i in range(21, 31):
        matrix[i, i-11] = 1
        matrix[i, i-10] = 1
        if i > 21 and i < 30:
            matrix[i, i-1] = 1
            matrix[i, i+1] = 1
        if i == 21:
            matrix[i, i+1] = 1

        if i == 30:
            matrix[i, i-1] = 1
        matrix[i, i+10] = 1
        matrix[i, i+11] = 1

    begin = 31
    end = 42

    for i in range(begin, end):
        if i > begin and i < end-1:
            matrix[i, i-11] = 1
            matrix[i, i-10] = 1
            matrix[i, i-1] = 1
            matrix[i, i+1] = 1
            matrix[i, i+10] = 1
            matrix[i, i+11] = 1
        if i == begin:
            matrix[i, i-10] = 1
            matrix[i, i+1] = 1
            matrix[i, i+11] = 1
        if i == end-1:
            matrix[i, i-11] = 1
            matrix[i, i-1] = 1
            matrix[i, i+10] = 1

    begin = 42
    end = 52

    for i in range(begin, end):
        if i > begin and i < end-1:
            matrix[i, i-11] = 1
            matrix[i, i-10] = 1
            matrix[i, i-1] = 1
            matrix[i, i+1] = 1
        if i == begin:
            matrix[i, i-11] = 1
            matrix[i, i-10] = 1
            matrix[i, i+1] = 1
        if i == end-1:
            matrix[i, i-11] = 1
            matrix[i, i-10] = 1
            matrix[i, i-1] = 1
    return matrix


In [6]:
# hbo_data = np.load('/Users/shanxiafeng/Documents/Code/python/fnirs_DL/JinyuanWang_pythonCode/allData/Output_npy/twoDoctor/HbO-All-HC-MDD/correct_channel_data.npy')
hbo_data = np.load('/Users/shanxiafeng/Documents/Code/python/fnirs_DL/JinyuanWang_pythonCode/allData/Output_npy/twoDoctor/HbO-All-HC-MDD/correct_channel_data.npy')

hbo_data = np.transpose(hbo_data, (0,2,1))
# hbr_data = np.load('/Users/shanxiafeng/Documents/Code/python/fnirs_DL/JinyuanWang_pythonCode/allData/Output_npy/twoDoctor/HbR-All-HC-MDD/correct_channel_data.npy')
hbr_data = np.load('/Users/shanxiafeng/Documents/Code/python/fnirs_DL/JinyuanWang_pythonCode/allData/Output_npy/twoDoctor/HbR-All-HC-MDD/correct_channel_data.npy')

hbr_data = np.transpose(hbr_data, (0,2,1))
labels = np.load('/Users/shanxiafeng/Documents/Code/python/fnirs_DL/JinyuanWang_pythonCode/allData/Output_npy/twoDoctor/HbO-All-HC-MDD/label.npy')
print(labels.shape)

# input = np.random.rand(458, 52, 125)
# please make sure the input data is normalized using something like z-norm
def normalize(data):
    # Iterate over each subject
    normalized_data = np.empty_like(data)
    # Calculate the mean and standard deviation for the current subject
    mean = np.mean(data)
    std = np.std(data)

    # Perform z-normalization for the current subject
    normalized_data= (data - mean) / std
    return normalized_data


# Concatenate the HbO: (, , 125); HbR: (, , 125) -> Con_HbO_HbR: (, , 250)
hb_input = normalize(np.concatenate((hbo_data,hbr_data),axis=2))

# read the multiview connectivity 

connectivity = np.load('/Users/shanxiafeng/Documents/Code/python/fnirs_DL/JinyuanWang_pythonCode/allData/data_for_reproducing_model/yu_gnn/A.npy') 
defined_neighbour = generate_fnirs_adj() 
defined_neighbour = defined_neighbour.toarray()

defined_neighbour = np.tile(defined_neighbour, (connectivity.shape[0], 1, 1))[..., np.newaxis]

adj = np.concatenate((defined_neighbour, connectivity), axis=3)

print(f'You have the hemolgobin hb_input: {hb_input.shape}')
print(f'You have the hemolgobin connectivity map: {adj.shape}')

(458,)
You have the hemolgobin hb_input: (458, 52, 250)
You have the hemolgobin connectivity map: (458, 52, 52, 5)


/Users/shanxiafeng/miniconda3/lib/python3.9/site-packages/scipy/sparse/_index.py:100: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


In [7]:
# Next, you will generate a simple GNN with mutliview aggregator using the hb_input and adj


You have the hemolgobin input: (458, 52, 250)
You have the hemolgobin connectivity map: (458, 52, 52, 5)


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import time

import tensorflow.keras.layers as kl
import tensorflow.keras.models as km
import tensorflow.keras.optimizers as ko
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
import tensorflow.keras as keras
from tensorflow.keras.initializers import HeNormal
from sklearn.metrics import auc, accuracy_score, roc_curve, recall_score
import tensorflow_addons as tfa
import random
import os
import pandas as pd
import math

from tensorflow.keras.callbacks import EarlyStopping

/Users/shanxiafeng/miniconda3/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/Users/shanxiafeng/miniconda3/lib/python3.9/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.11.0 and strictly below 2.14.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.14.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issu

In [30]:
import tensorflow as tf

class MultiViewAggregator(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(MultiViewAggregator, self).__init__(**kwargs)

    def build(self, input_shape):
        # Assuming input_shape is (subjects, node_seed, node_target, number_of_views)
        self.num_views = input_shape[-1]
        feat_dim = input_shape[-2]  # node_target dimension is considered as feature dimension

        # This is the learnable weight vector b̅ mentioned in the formula (6)
        self.weight_vector = self.add_weight(shape=(feat_dim,),
                                             initializer='glorot_uniform',
                                             name='weight_vector')

    def call(self, inputs):
        print(f'inputs -> {inputs.shape}')
        # inputs shape is (subjects, node_seed, node_target, number_of_views)
        # Compute the dot product between weight vector and node_target features for each view
        attention_scores = tf.tensordot(inputs, self.weight_vector, axes=[2, 0])
        
        # attention_scores shape is (subjects, node_seed, number_of_views)
        # Apply softmax over the number_of_views dimension to get the attention coefficients
        attention_coefficients = tf.nn.softmax(attention_scores, axis=-1)

        # Expand dims of attention_coefficients for element-wise multiplication
        attention_coefficients_expanded = tf.expand_dims(attention_coefficients, axis=2)

        # Multiply the inputs with the attention coefficients and sum over the views
        weighted_sum = tf.reduce_sum(inputs * attention_coefficients_expanded, axis=-1)

        return weighted_sum

    def compute_output_shape(self, input_shape):
        # The output shape will be the input shape without the number_of_views dimension
        return input_shape[:-1]

# Test the MultiViewAggregator
# Define the input shape parameters
num_subjects = 10
num_node_seed = 5
num_node_target = 5
num_views = 4

# Generate random data to simulate the inputs
inputs = tf.random.normal((num_subjects, num_node_seed, num_node_target, num_views))

# Create the MultiViewAggregator layer
aggregator = MultiViewAggregator()

# Call the aggregator on the test inputs
aggregated_representation = aggregator(inputs)

# Print the shapes to verify the output
print(f"Shape of inputs: {inputs.shape}")
print(f"Shape of aggregated representation: {aggregated_representation.shape}")
# Output the actual aggregated representation for inspection
print(aggregated_representation)

inputs -> (10, 5, 5, 4)
Shape of inputs: (10, 5, 5, 4)
Shape of aggregated representation: (10, 5, 5)
tf.Tensor(
[[[ 4.44647729e-01  4.85640883e-01  5.44205606e-01 -4.87229794e-01
   -1.54046500e+00]
  [-9.19233263e-01  1.47613156e+00  1.12740993e-02 -1.01495934e+00
   -1.00478105e-01]
  [-4.06379141e-02  6.16269484e-02  4.93018925e-01 -3.81188095e-01
   -3.99531797e-02]
  [-4.52260762e-01  3.92010748e-01 -9.12942886e-01 -9.09008384e-02
    8.06296527e-01]
  [ 1.12404704e+00 -7.51969099e-01  4.27148670e-01  9.20496583e-02
   -8.58191013e-01]]

 [[-1.04435861e-01  2.95010805e-01 -9.47994351e-01 -2.05423772e-01
    2.59557605e-01]
  [-2.63141632e-01 -5.50736487e-01 -2.08856583e+00  7.89493680e-01
    5.01750648e-01]
  [-3.95877272e-01  1.66038930e-01 -3.56380761e-01 -2.13012531e-01
   -1.42559659e+00]
  [-8.10113668e-01 -1.92861128e+00 -1.76375306e+00  2.99217671e-01
   -1.00809336e+00]
  [-9.16985154e-01 -3.41886222e-01 -6.28981888e-02 -3.37504327e-01
   -2.26181269e+00]]

 [[-7.5422257

In [49]:
a = tf.ones((num_subjects, num_node_seed, num_node_target, num_views))
print(a.shape)
b = tf.constant([1,2,3,4,5], dtype=tf.float32)
# print(b)

c = tf.tensordot(a,b, axes=[2,0])
print(c.shape)

(10, 5, 5, 4)
(10, 5, 4)


In [7]:


class MultiViewAggregator(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(MultiViewAggregator, self).__init__(**kwargs)

    def build(self, input_shape):
        # Assuming input_shape is (subjects, node_seed, node_target, number_of_views)
        self.num_views = input_shape[-1]
        feat_dim = input_shape[-2]  # node_target dimension is considered as feature dimension

        # This is the learnable weight vector b̅ mentioned in the formula (6)
        self.weight_vector = self.add_weight(shape=(self.num_views,),
                                             initializer='glorot_uniform',
                                             name='weight_vector')

    def call(self, inputs):
        print(f'inputs -> {inputs.shape}')
        # inputs shape is (subjects, node_seed, node_target, number_of_views)
        # Compute the dot product between weight vector and node_target features for each view
        attention_scores = tf.tensordot(inputs, self.weight_vector, axes=[3, 0])
        print(f'attention_scores: {attention_scores.shape}')
        # attention_scores shape is (subjects, node_seed, number_of_views)
        # Apply softmax over the number_of_views dimension to get the attention coefficients
        attention_coefficients = tf.nn.softmax(attention_scores, axis=-1)
        print(f'attention_coefficients: {attention_coefficients.shape}')
        # Expand dims of attention_coefficients for element-wise multiplication
        attention_coefficients_expanded = tf.expand_dims(attention_coefficients, axis=-1)
        print(f'attention_coefficients_expanded: {attention_coefficients_expanded.shape}')

        # Multiply the inputs with the attention coefficients and sum over the views
        weighted_sum = tf.reduce_sum(inputs * attention_coefficients_expanded, axis=-1)
        print(f'weighted_sum: {weighted_sum.shape}')

        return weighted_sum

    def compute_output_shape(self, input_shape):
        # The output shape will be the input shape without the number_of_views dimension
        return input_shape[:-1]
    
    
class MVGCN(tf.keras.Model):
    def __init__(self,
                 d_model,
                 activation='relu'):
        super(MVGCN, self).__init__()

        self.W = layers.Conv2D(filters=1,
                                   kernel_size=[1, 3])
        # self.W = layers.Dense(units=d_model)
        self.aggregator = MultiViewAggregator()

        if activation == 'relu':
            self.activation = tf.keras.layers.ReLU()
        elif activation == 'sigmoid':
            self.activation = tf.keras.layers.Activation('sigmoid')
        elif activation == 'tanh':
            self.activation = tf.keras.layers.Activation('tanh')
        elif activation == 'prelu':
            self.activation = tf.keras.layers.Activation('prelu')
        else:
            raise ValueError('Provide a valid activation for GNN')
        

    def normalize_adjacency(self, adj):
        adj = self.aggregator(adj)
        d = tf.reduce_sum(adj, axis=-1)
        d_sqrt_inv = tf.pow(d, -0.5)
        d_sqrt_inv = tf.where(tf.math.is_inf(d_sqrt_inv), 0., d_sqrt_inv)
        d_mat_inv_sqrt = tf.linalg.diag(d_sqrt_inv)
        return tf.matmul(tf.matmul(d_mat_inv_sqrt, adj), d_mat_inv_sqrt)

    def call(self, inputs, adj):
        adj_normalized = self.normalize_adjacency(adj)
        inputs = inputs[..., tf.newaxis]
        inputs_features = self.W(inputs)
        inputs_features = tf.squeeze(inputs_features, axis=-1)
        print(f'xxx inputs_features -> {inputs_features.shape}')
        outputs = tf.linalg.matmul(adj_normalized, inputs_features)
        return self.activation(outputs)



class Classifier_MVG_Transformer():
    def __init__(self, output_directory, callbacks, input_shape, epochs, sweep_config, info):
        # input_shape = (200, 52, 128, 1)

        self.epochs = epochs

        # 随机给定超参数进行训练
        self.info = info
        self.batch_size = sweep_config['batch_size'] if sweep_config else 128
        d_model = 64  # 125# # random.choice([64, 128, 256])
        adam_beta_1, adam_beta_2 = 0.9, 0.999
        num_class = 2  # 2
        learning_rate = 0.001
        num_of_view = 5 
        optimizer = tf.keras.optimizers.AdamW(learning_rate,
                                              beta_1=adam_beta_1,
                                              beta_2=adam_beta_2,
                                              epsilon=1e-9)

        # If you change these two hyperparameters, remember to change the  self.hyperparameters
        inputs = tf.keras.Input(shape=input_shape[1:])
        input_adj = tf.keras.Input(shape=(input_shape[1], input_shape[1], num_of_view))
        
        x = layers.Dense(64)(inputs)
        outputs = MVGCN(d_model=d_model)(x, input_adj)
        outputs = MVGCN(d_model=d_model)(outputs, input_adj)
        outputs = layers.Dense(units=d_model)(outputs)
        outputs = layers.GlobalAveragePooling1D(
            data_format='channels_first', keepdims=False)(outputs)
        outputs = layers.Dense(num_class, activation='softmax')(outputs)
        model = tf.keras.Model(inputs=[inputs, input_adj], outputs=outputs)
        model.summary()
        model.compile(optimizer=optimizer,
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        self.model = model

    def fit(self, X_train, Y_train, X_val, Y_val, X_test, Y_test, adj_train, adj_val, adj_test):
        hist = self.model.fit(
            x=[X_train, adj_train],
            y=Y_train,
            validation_data=([X_val, adj_val], Y_val),
            batch_size=self.batch_size,
            epochs=self.epochs,
            verbose=True,
            shuffle=True  # Set shuffle to True
        )
epochs = 1000
classifer = Classifier_MVG_Transformer(None, None, hb_input.shape, epochs=epochs, sweep_config=None, info=None)

def onehotEncode(x):
    t = np.zeros((x.size, x.max()+1))
    t[np.arange(x.size), x] = 1
    return t.astype(int)
onehot_labels = onehotEncode(labels)

X_train, Y_train = hb_input[:350], onehot_labels[:350]
X_val, Y_val = hb_input[350:400], onehot_labels[350:400]
X_test, Y_test = hb_input[400:], onehot_labels[400:]
adj_train, adj_val, adj_test = adj[:350], adj[350:400], adj[400:]

classifer.fit(X_train, Y_train, X_val, Y_val, X_test, Y_test, adj_train, adj_val, adj_test)

inputs -> (None, 52, 52, 5)
attention_scores: (None, 52, 52)
attention_coefficients: (None, 52, 52)
attention_coefficients_expanded: (None, 52, 52, 1)
weighted_sum: (None, 52, 52)
xxx inputs_features -> (None, 52, 62)
inputs -> (None, 52, 52, 5)
attention_scores: (None, 52, 52)
attention_coefficients: (None, 52, 52)
attention_coefficients_expanded: (None, 52, 52, 1)
weighted_sum: (None, 52, 52)
xxx inputs_features -> (None, 52, 60)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 52, 250)]            0         []                            
                                                                                                  
 dense (Dense)               (None, 52, 64)               16064     ['input_1[0][0]']             
                                                      

KeyboardInterrupt: 

In [95]:
# CNN -> GNN -> Transfromer 


class MultiChannelCNN(tf.keras.layers.Layer):
    def __init__(self, kernel_size, padding, activation, **kwargs):
        super(MultiChannelCNN, self).__init__(**kwargs)
        self.kernel_size = kernel_size
        self.padding = padding
        self.activation = activation
        self.conv1 = kl.Conv1D(filters=32, kernel_size=kernel_size[0], padding=padding, kernel_initializer=HeNormal())
        self.conv2 = kl.Conv1D(filters=32, kernel_size=kernel_size[1], padding=padding, kernel_initializer=HeNormal())
        self.conv3 = kl.Conv1D(filters=1, kernel_size=kernel_size[1], padding=padding, kernel_initializer=HeNormal())
        self.flatten = kl.Flatten()
        self.dense = kl.Dense(1, activation=activation)

    def call(self, inputs):
        # Use a TensorArray for dynamically storing the outputs
        all_m = []
        for i in range(inputs.shape[1]):
            channel = inputs[:, i, :, :]
            m0 = self.conv1(channel)
            m0 = kl.Activation(self.activation)(m0)

            m0 = self.conv2(m0)  # Reuse the same Conv1D layer
            m0 = kl.Activation(self.activation)(m0)

            m0 = self.conv3(m0)
            m0 = kl.Activation(self.activation)(m0)
            print(f'test before flatten m0.shape: {m0.shape}')
            m0 = self.flatten(m0)
            print(f'test after flatten m0.shape: {m0.shape}')
            # m0 = self.dense(m0)
            all_m.append(m0)

        output = tf.stack(all_m, axis=1)
        
        output = tf.transpose(output, perm=[0, 1, 2]) 
        print(f'output -> {output.shape}')
        return output


class MultiViewAggregator(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(MultiViewAggregator, self).__init__(**kwargs)

    def build(self, input_shape):
        # Assuming input_shape is (subjects, node_seed, node_target, number_of_views)
        self.num_views = input_shape[-1]
        feat_dim = input_shape[-2]  # node_target dimension is considered as feature dimension

        # This is the learnable weight vector b̅ mentioned in the formula (6)
        self.weight_vector = self.add_weight(shape=(self.num_views,),
                                             initializer='glorot_uniform',
                                             name='weight_vector')

    def call(self, inputs):
        print(f'inputs -> {inputs.shape}')
        # inputs shape is (subjects, node_seed, node_target, number_of_views)
        # Compute the dot product between weight vector and node_target features for each view
        attention_scores = tf.tensordot(inputs, self.weight_vector, axes=[3, 0])
        print(f'attention_scores: {attention_scores.shape}')
        # attention_scores shape is (subjects, node_seed, number_of_views)
        # Apply softmax over the number_of_views dimension to get the attention coefficients
        attention_coefficients = tf.nn.softmax(attention_scores, axis=-1)
        print(f'attention_coefficients: {attention_coefficients.shape}')
        # Expand dims of attention_coefficients for element-wise multiplication
        attention_coefficients_expanded = tf.expand_dims(attention_coefficients, axis=-1)
        print(f'attention_coefficients_expanded: {attention_coefficients_expanded.shape}')

        # Multiply the inputs with the attention coefficients and sum over the views
        weighted_sum = tf.reduce_sum(inputs * attention_coefficients_expanded, axis=-1)
        print(f'weighted_sum: {weighted_sum.shape}')

        return weighted_sum

    def compute_output_shape(self, input_shape):
        # The output shape will be the input shape without the number_of_views dimension
        return input_shape[:-1]
    
class MVGCN(tf.keras.Model):
    def __init__(self,
                 d_model,
                 activation='relu'):
        super(MVGCN, self).__init__()

        self.W = layers.Conv2D(filters=1,
                                   kernel_size=[1, 3])
        # self.W = layers.Dense(units=d_model)
        self.aggregator = MultiViewAggregator()

        if activation == 'relu':
            self.activation = tf.keras.layers.ReLU()
        elif activation == 'sigmoid':
            self.activation = tf.keras.layers.Activation('sigmoid')
        elif activation == 'tanh':
            self.activation = tf.keras.layers.Activation('tanh')
        elif activation == 'prelu':
            self.activation = tf.keras.layers.Activation('prelu')
        else:
            raise ValueError('Provide a valid activation for GNN')
        

    def normalize_adjacency(self, adj):
        adj = self.aggregator(adj)
        d = tf.reduce_sum(adj, axis=-1)
        d_sqrt_inv = tf.pow(d, -0.5)
        d_sqrt_inv = tf.where(tf.math.is_inf(d_sqrt_inv), 0., d_sqrt_inv)
        d_mat_inv_sqrt = tf.linalg.diag(d_sqrt_inv)
        return tf.matmul(tf.matmul(d_mat_inv_sqrt, adj), d_mat_inv_sqrt)

    def call(self, inputs, adj):
        adj_normalized = self.normalize_adjacency(adj)
        inputs = inputs[..., tf.newaxis]
        inputs_features = self.W(inputs)
        inputs_features = tf.squeeze(inputs_features, axis=-1)
        print(f'xxx inputs_features -> {inputs_features.shape}')
        outputs = tf.linalg.matmul(adj_normalized, inputs_features)
        return self.activation(outputs)

class Classifier_MVG_Transformer():
    def __init__(self, output_directory, callbacks, input_shape, epochs, sweep_config, info):
        # input_shape = (200, 52, 128, 1)

        self.epochs = epochs

        # 随机给定超参数进行训练
        self.info = info
        self.batch_size = sweep_config['batch_size'] if sweep_config else 128
        d_model = 64  # 125# # random.choice([64, 128, 256])
        adam_beta_1, adam_beta_2 = 0.9, 0.999
        num_class = 2  # 2
        learning_rate = 0.001
        num_of_view = 5 
        optimizer = tf.keras.optimizers.AdamW(learning_rate,
                                              beta_1=adam_beta_1,
                                              beta_2=adam_beta_2,
                                              epsilon=1e-9)

        # If you change these two hyperparameters, remember to change the  self.hyperparameters
        inputs = tf.keras.Input(shape=[input_shape[1], input_shape[2]])
        input_adj = tf.keras.Input(shape=(input_shape[1], input_shape[1], num_of_view))
        outputs = MultiChannelCNN(padding= 'same', kernel_size= [8, 3], activation='sigmoid')(inputs[..., tf.newaxis])
        print(f'outputs MVG -> {outputs.shape}')
        outputs = MVGCN(d_model=d_model)(inputs, input_adj)
        outputs = MVGCN(d_model=d_model)(outputs, input_adj)
        outputs = layers.Dense(units=d_model)(outputs)
        outputs = layers.GlobalAveragePooling1D(
            data_format='channels_first', keepdims=False)(outputs)
        outputs = layers.Dense(num_class, activation='softmax')(outputs)
        model = tf.keras.Model(inputs=[inputs, input_adj], outputs=outputs)
        model.summary()
        model.compile(optimizer=optimizer,
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        self.model = model

    def fit(self, X_train, Y_train, X_val, Y_val, X_test, Y_test, adj_train, adj_val, adj_test):
        hist = self.model.fit(
            x=[X_train, adj_train],
            y=Y_train,
            validation_data=([X_val, adj_val], Y_val),
            batch_size=self.batch_size,
            epochs=self.epochs,
            verbose=True,
            shuffle=True  # Set shuffle to True
        )
epochs = 1000
classifer = Classifier_MVG_Transformer(None, None, hb_input.shape, epochs=epochs, sweep_config=None, info=None)

def onehotEncode(x):
    t = np.zeros((x.size, x.max()+1))
    t[np.arange(x.size), x] = 1
    return t.astype(int)
onehot_labels = onehotEncode(labels)

X_train, Y_train = hb_input[:350], onehot_labels[:350]
X_val, Y_val = hb_input[350:400], onehot_labels[350:400]
X_test, Y_test = hb_input[400:], onehot_labels[400:]
adj_train, adj_val, adj_test = adj[:350], adj[350:400], adj[400:]

classifer.fit(X_train, Y_train, X_val, Y_val, X_test, Y_test, adj_train, adj_val, adj_test)

test before flatten m0.shape: (None, 250, 1)
test after flatten m0.shape: (None, 250)
test before flatten m0.shape: (None, 250, 1)
test after flatten m0.shape: (None, 250)
test before flatten m0.shape: (None, 250, 1)
test after flatten m0.shape: (None, 250)
test before flatten m0.shape: (None, 250, 1)
test after flatten m0.shape: (None, 250)
test before flatten m0.shape: (None, 250, 1)
test after flatten m0.shape: (None, 250)
test before flatten m0.shape: (None, 250, 1)
test after flatten m0.shape: (None, 250)
test before flatten m0.shape: (None, 250, 1)
test after flatten m0.shape: (None, 250)
test before flatten m0.shape: (None, 250, 1)
test after flatten m0.shape: (None, 250)
test before flatten m0.shape: (None, 250, 1)
test after flatten m0.shape: (None, 250)
test before flatten m0.shape: (None, 250, 1)
test after flatten m0.shape: (None, 250)
test before flatten m0.shape: (None, 250, 1)
test after flatten m0.shape: (None, 250)
test before flatten m0.shape: (None, 250, 1)
test afte

KeyboardInterrupt: 

NameError: name 'input_shape' is not defined